In [13]:
from selenium import webdriver
from selenium.webdriver.common import actions
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as ec
import pandas as pd 
import os 
import sys
from datetime import date, datetime
import chromedriver_autoinstaller
from bs4 import BeautifulSoup
import lxml
import time

In [14]:
def clicking(path, element='elemento selecionado', refresh=False, by=By.XPATH, limit=3):
    """
    * path: caminho do elemento que será encontrado na página. Pode ser um xpath, um ID, um name e etc;
    * element: nome do elemento que será encontrado na página. Esse parâmetro tem como finalidade unicamente para melhor depuração. Pode ser omitido;
    * refresh: default False - tem como objetivo atualizar a página para tentar clicar no elemento. Utilizado para clicar em arquivos em listas de query;
    * by: default XPATH - método utilizado para localizar o elemento, como XPATH, ID, NAME, CSS_SELECTOR e etc;
    * limit: default 3 - quantidade de vezes que a função irá tentar clicar no elemento. Por padrão, cada tentativa leva 20 segundos. Se o elemento não for encontrado, retornará None;
    """


    if limit == 0:
        return None


    elif refresh:
        driver.refresh()

    try:
        object = driver.find_element(by, path)
        actions = ActionChains(driver)
        actions.move_to_element(object).perform()
        
        found_element = driver.find_element(by, path)
        print(f">>>Sucesso {element}>>>")

    except:
        print(f"<<<Erro ao {element}, tentando novamente>>>")
        found_element = clicking(element=element, path=path, refresh=refresh, by=by, limit=limit - 1)


    return found_element

def create_driver(download_dir=None, headless=False):
    chrome_options = Options()

    if headless:
        chrome_options.headless=True

    if download_dir is not None:
        download_dir=str(download_dir)


    preferences = {"download.default_directory": download_dir,
                   "directory_upgrade": True,
                   "safebrowsing.enabled": True}
    chrome_options.add_experimental_option("prefs", preferences)
    chromedriver_autoinstaller.install()
    driver = webdriver.Chrome(options=chrome_options)
    driver.implicitly_wait(20)

    return driver

def error(e):
    print("Apresentou erro, gravando o erro")
    exc_type, exc_obj, exc_tb = sys.exc_info()
    fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
    path_erro = r'\\branas06av3-nrd\LAR_BRA_DIGITAL_F\Facens\Logs'
    #path_erro = r"C:\Users\IWX1057086\Documents\Programas\TIM RPA\log"
    today = date.today()
    name = str(fname) + '_' + str(today) + '.txt'
    arquivo = open(path_erro + '\\' + name, 'w')
    arquivo.write(str(fname) + "\n")
    arquivo.write(str(e) + "\n")
    arquivo.write(str(exc_type) + "\n")
    arquivo.write(str(exc_tb.tb_lineno) + "\n")
    arquivo.close()
   

In [15]:
driver = create_driver(download_dir=None, headless=False)
driver.get('http://web.whatsapp.com/')

In [43]:
EXTRACT_SESSION = '''

function getResultFromRequest(request) {
    return new Promise((resolve, reject) => {
        request.onsuccess = function (event) {
            resolve(request.result);
        };
    });
}
   
async function getDB() {
    var request = window.indexedDB.open("wawc");
    return await getResultFromRequest(request);
}
   
async function readAllKeyValuePairs() {
    var db = await getDB();
    var objectStore = db.transaction("user").objectStore("user");
    var request = objectStore.getAll();
       return await getResultFromRequest(request);
}

async function injectSession(SESSION_STRING) {
    var session = JSON.parse(SESSION_STRING);
    var db = await getDB();
    var objectStore = db.transaction("user", "readwrite").objectStore("user");
    for(var keyValue of session) {
        var request = objectStore.put(keyValue);
        await getResultFromRequest(request);
    }
}
   

session = await readAllKeyValuePairs();
console.log(session);

var SESSION_STRING = JSON.stringify(session);
console.log(SESSION_STRING);

await injectSession();
'''


In [44]:
session = driver.execute_script(EXTRACT_SESSION)

JavascriptException: Message: javascript error: Unexpected token u in JSON at position 0
  (Session info: chrome=103.0.5060.114)
Stacktrace:
Backtrace:
	Ordinal0 [0x00A256B3+2184883]
	Ordinal0 [0x009BE5F1+1762801]
	Ordinal0 [0x008D3DA8+802216]
	Ordinal0 [0x008D6784+812932]
	Ordinal0 [0x008D6642+812610]
	Ordinal0 [0x008D6FBA+815034]
	Ordinal0 [0x0092D989+1169801]
	Ordinal0 [0x0091CB0C+1100556]
	Ordinal0 [0x0092CF52+1167186]
	Ordinal0 [0x0091C926+1100070]
	Ordinal0 [0x008F6EA0+945824]
	Ordinal0 [0x008F7D96+949654]
	GetHandleVerifier [0x00CC4192+2704034]
	GetHandleVerifier [0x00CB687D+2648461]
	GetHandleVerifier [0x00AB119A+529066]
	GetHandleVerifier [0x00AB0006+524566]
	Ordinal0 [0x009C4F2B+1789739]
	Ordinal0 [0x009C9978+1808760]
	Ordinal0 [0x009C9A65+1808997]
	Ordinal0 [0x009D2A01+1845761]
	BaseThreadInitThunk [0x767DFA29+25]
	RtlGetAppContainerNamedObjectPath [0x770B7A9E+286]
	RtlGetAppContainerNamedObjectPath [0x770B7A6E+238]


In [37]:
session

In [38]:
sessionFilePath = r'C:\Users\vitor\Documents\Programas\Bot_Controle_Loja\session.wa'

In [39]:
with open(sessionFilePath, "w", encoding="utf-8") as sessionFile:
    sessionFile.write(str(session))